# Setup Environment

In [ ]:
!pip install ultralytics

In [ ]:
from ultralytics import YOLO
import yaml


# Data Preparing

In [ ]:
# Function to create data.yaml file
def create_data_yaml(train_path, val_path, num_classes, class_names, output_path='data.yaml'):
    with open(output_path, 'w') as f:
        f.write(f"train: {train_path}\n")
        f.write(f"val: {val_path}\n")
        f.write(f"nc: {num_classes}\n")
        f.write(f"names: {class_names}\n")
    print(f"data.yaml created at {output_path}")

# User input for dataset paths and class names
train_path = "License-Plates.v1i.yolov8/train/images"
val_path = "License-Plates.v1i.yolov8/valid/images"
num_classes = int("1")
class_names = "license_plate".split(',')

# Clean up class names (strip whitespace)
class_names = [name.strip() for name in class_names]

# Create data.yaml in the current directory
create_data_yaml(train_path, val_path, num_classes, class_names, output_path='data.yaml')

# Path to your data.yaml file
yaml_path = 'data.yaml'

# Read the existing file
with open(yaml_path, 'r') as f:
    data = f.read()
    print(data)  # Display the current content


# Model Fine-Tuning

In [ ]:
# Import the necessary libraries

# Load the pre-trained YOLOv8 model for license plate detection
model = YOLO('/content/license_plate_detector.pt')

# Load your custom dataset
# Make sure your dataset is in YOLO format and located in 'path/to/your/dataset'
data = '/content/data.yaml'  # Adjust the path to your dataset configuration file

# Fine-tune the model
model.train(data=data, epochs=5, imgsz=100, batch=32)

# Save the fine-tuned model
model.save('fine_tuned_linecne_plate.pt')


# Model Evaluate

In [ ]:
results = model.val(data='/content/vehicle-and-license-plate-3/data.yaml', imgsz=640)

# Access and print the results using the results_dict attribute
results_dict = results.results_dict
print(f"Precision: {results_dict['metrics/precision(B)']:.4f}")
print(f"Recall: {results_dict['metrics/recall(B)']:.4f}")
print(f"mAP@0.5: {results_dict['metrics/mAP50(B)']:.4f}")
print(f"mAP@0.5:0.95: {results_dict['metrics/mAP50-95(B)']:.4f}")
print(f"F1 Score: {2 * (results_dict['metrics/precision(B)'] * results_dict['metrics/recall(B)']) / (results_dict['metrics/precision(B)'] + results_dict['metrics/recall(B)']):.4f}")